#**MAESTRÍA EN ANÁLISIS DE DATOS Y SISTEMAS INTELIGENTES**

## **ESTUDIO COMPARATIVO DE MODELOS DE APRENDIZAJE SUPERVISADO**


*   Edwar David Macías López
*   Miguel Andres Arias Romero
*   Javier Santiago Hernandez Mendez
*   Jhon Freddy Hernandez Corzo

#**Alistamiento y limpieza de datos**

En primer lugar se realiza el import de la libreria **Pandas**, se leen los datos del data-set y posteriormente se almacenan en el dataframe **df**.

Se hace una vista rápida de los datos para revisar nombres de columnas, tipos de valores y si hay errores evidentes, en los 5 primeros registros:

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/EdwMacias/siniestrosBogota/refs/heads/main/SiniestrosBog_DataSet.csv"
df = pd.read_csv(url)

Ahora se valida la información general del DataFrame, se verifican los nombres de columnas, cantidad de valores no nulos, tipo de dato de cada columna (object, int64, float64, etc.) y número total de filas:

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196152 entries, 0 to 196151
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   FECHA        196152 non-null  object 
 1   HORA         196152 non-null  object 
 2   LOCALIDAD    196152 non-null  object 
 3   GRAVEDAD     196152 non-null  object 
 4   CLASE        196152 non-null  object 
 5   CHOQUE       167910 non-null  float64
 6   OBJETO_FIJO  6689 non-null    float64
 7   DIRECCION    196152 non-null  object 
 8   ACTOR VIAL   196004 non-null  object 
dtypes: float64(2), object(7)
memory usage: 13.5+ MB


También se realiza la validación de las muestra estadísticas descriptivas de las columnas numéricas por defecto.

Se verifica la Media, Desviación estándar, Mínimo, Máximo y Cuartiles (25%, 50%, 75%):

In [3]:
df.describe()

,CHOQUE,OBJETO_FIJO
count,167910.000000,6689.000000
mean,1.127717,5.318583
std,0.610152,3.756058
min,1.000000,1.000000
25%,1.000000,2.000000
50%,1.000000,5.000000
75%,1.000000,10.000000
max,5.000000,16.000000


Se verifican cuántos valores nulos (NaN) hay por columna:

In [4]:
df.isnull().sum()

FECHA               0
HORA                0
LOCALIDAD           0
GRAVEDAD            0
CLASE               0
CHOQUE          28242
OBJETO_FIJO    189463
DIRECCION           0
ACTOR VIAL        148
dtype: int64

Y también cuántas filas duplicadas hay en el DataFrame:

In [5]:
df.duplicated().sum()

np.int64(65)

Se eliminan las columnas poco representativas para el ejericio y los valores duplicados.

Se visualiza de nuevo el DataFrame:

In [6]:
df.drop(columns=['DIRECCION', 'CHOQUE', 'OBJETO_FIJO'], inplace=True)
df.drop_duplicates(inplace=True)
df = df.dropna()
df.head()

,FECHA,HORA,LOCALIDAD,GRAVEDAD,CLASE,ACTOR VIAL
0,1/1/2015,01:05:00,Puente Aranda,Con Heridos,Atropello,CONDUCTOR
1,1/1/2015,05:50:00,Bosa,Con Heridos,Volcamiento,MOTOCICLISTA
2,1/1/2015,07:15:00,Ciudad Bolívar,Con Heridos,Volcamiento,MOTOCICLISTA
3,1/1/2015,09:30:00,Kennedy,Solo Daños,Choque,CONDUCTOR
4,1/1/2015,09:45:00,Engativá,Con Heridos,Choque,CONDUCTOR


Ahora se realiza la codificación de la variable objetivo GRAVEDAD, para lo cual se crea la columna codificada GRAVEDAD_COD:

In [7]:
gravedad_map = {
    'Solo Daños': 0,
    'Con Heridos': 1,
    'Con Muertos': 2
}
df['GRAVEDAD_COD'] = df['GRAVEDAD'].map(gravedad_map)

Otra transformación que se puede aplicar es extraer la hora numérica de la columna HORA, con el fin de mediante una función, categorizar las horas de los accidentes en franjas y crear la columna HORA_FRANJA:

In [8]:
df['HORA_NUM'] = df['HORA'].str.extract(r'(\d{1,2})').astype(float)

def categorizar_franja(hora):
    if 5 <= hora < 10:
        return 'Madrugada'
    elif 10 <= hora < 14:
        return 'Mañana'
    elif 14 <= hora < 18:
        return 'Tarde'
    elif 18 <= hora < 22:
        return 'Noche'
    else:
        return 'Madrugada Tarde'

df['HORA_FRANJA'] = df['HORA_NUM'].apply(categorizar_franja)


Se realiza la transformación de variables de tiempo y se procede a crear columna DIA_SEMANA:

In [9]:
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
df['DIA_SEMANA'] = df['FECHA'].dt.day_name()

df.head()

,FECHA,HORA,LOCALIDAD,GRAVEDAD,CLASE,ACTOR VIAL,GRAVEDAD_COD,HORA_NUM,HORA_FRANJA,DIA_SEMANA
0,2015-01-01,01:05:00,Puente Aranda,Con Heridos,Atropello,CONDUCTOR,1,1.0,Madrugada Tarde,Thursday
1,2015-01-01,05:50:00,Bosa,Con Heridos,Volcamiento,MOTOCICLISTA,1,5.0,Madrugada,Thursday
2,2015-01-01,07:15:00,Ciudad Bolívar,Con Heridos,Volcamiento,MOTOCICLISTA,1,7.0,Madrugada,Thursday
3,2015-01-01,09:30:00,Kennedy,Solo Daños,Choque,CONDUCTOR,0,9.0,Madrugada,Thursday
4,2015-01-01,09:45:00,Engativá,Con Heridos,Choque,CONDUCTOR,1,9.0,Madrugada,Thursday


Posteriormente se identifican y codifican las variables categóricas.  Se podría usar **One Hot Encoding** pero en este caso se realiza la codificación con **Ordinal Encoding** para tener un enfoque más simple manteniendo las columnas con valores numéricos y sin aumentar dimensionalidad:

In [10]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
df[['LOCALIDAD', 'CLASE', 'ACTOR VIAL', 'HORA_FRANJA', 'DIA_SEMANA']] = encoder.fit_transform(df[['LOCALIDAD', 'CLASE', 'ACTOR VIAL', 'HORA_FRANJA', 'DIA_SEMANA']])

df.head()

,FECHA,HORA,LOCALIDAD,GRAVEDAD,CLASE,ACTOR VIAL,GRAVEDAD_COD,HORA_NUM,HORA_FRANJA,DIA_SEMANA
0,2015-01-01,01:05:00,10.0,Con Heridos,0.0,1.0,1,1.0,1.0,4.0
1,2015-01-01,05:50:00,2.0,Con Heridos,6.0,2.0,1,5.0,0.0,4.0
2,2015-01-01,07:15:00,4.0,Con Heridos,6.0,2.0,1,7.0,0.0,4.0
3,2015-01-01,09:30:00,7.0,Solo Daños,3.0,1.0,0,9.0,0.0,4.0
4,2015-01-01,09:45:00,5.0,Con Heridos,3.0,1.0,1,9.0,0.0,4.0


Se validan los resultado:

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194035 entries, 0 to 196151
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   FECHA         194035 non-null  datetime64[ns]
 1   HORA          194035 non-null  object        
 2   LOCALIDAD     194035 non-null  float64       
 3   GRAVEDAD      194035 non-null  object        
 4   CLASE         194035 non-null  float64       
 5   ACTOR VIAL    194035 non-null  float64       
 6   GRAVEDAD_COD  194035 non-null  int64         
 7   HORA_NUM      194035 non-null  float64       
 8   HORA_FRANJA   194035 non-null  float64       
 9   DIA_SEMANA    194035 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(1), object(2)
memory usage: 16.3+ MB


# **Modelo de Clasificación**

 Teniendo los datos limpios y estructurados, se aplicaran las tecnicas de clasificación, usando como **variable objetivo** la Gravedad (GRAVEDAD_COD: 0 = Solo Daños, 1 = Con Heridos, 2 = Con Muertos):

 1. Se procede a separar la varible objetivo:

In [12]:
y = df['GRAVEDAD_COD']
y.head()

0    1
1    1
2    1
3    0
4    1
Name: GRAVEDAD_COD, dtype: int64

2. De las demás variables (variables predictoras) se excluyen las varibles que no serán de mucha utilidad:

In [13]:
X = df.drop(columns=['GRAVEDAD', 'GRAVEDAD_COD', 'FECHA', 'HORA'])
X.head()

,LOCALIDAD,CLASE,ACTOR VIAL,HORA_NUM,HORA_FRANJA,DIA_SEMANA
0,10.0,0.0,1.0,1.0,1.0,4.0
1,2.0,6.0,2.0,5.0,0.0,4.0
2,4.0,6.0,2.0,7.0,0.0,4.0
3,7.0,3.0,1.0,9.0,0.0,4.0
4,5.0,3.0,1.0,9.0,0.0,4.0


3. Se dividen los datos en los conjuntos de entrenamiento (train) y prueba (test), así:

*   Se define para la varible **test_size=0.2** lo que permitirá que el 20% de los datos se utilicen para prueba, y el 80% para entrenamiento.
*   Se define el valor de **random_state=42** para garantiza que la división sea reproducible cada vez que ejecutes el código.
*   Adicionalmente se define el valor de **stratify=y** en y, para que se mantengan la proporción de clases de la variable objetivo (y) tanto en el conjunto de entrenamiento como en el de prueba.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

4. Entrenamiento del modelo a partir de diferentes algoritmos:

**Árbol de decisión:**

Se define la importación del modelo **DecisionTreeClassifier** de la llibreria **sklearn.tree** y posteriormente se crea el objeto con los siguientes parámetros personalizados:

*   **max_depth=5**: Para limitar la profundidad del árbol a 5 niveles.
*   **random_state=42**: Semilla aleatoria fija para que el resultado sea reproducible.
*   **class_weight='balanced'**: Para corregir el desequilibrio en las clases.

Posteriormente se entrena el modelo:



In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

modelo_tree = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')

modelo_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)

Ahora se hacen predicciones con el modelo sobre los datos de prueba ***X_test*** y se evalúa el rendimiento del modelo de clasificación, mediante tres métricas importantes: accuracy, matriz de confusión y reporte de clasificación:

**accuracy_score:** Se calcula la exactitud, es decir, el porcentaje de predicciones correctas, con dos decimales.

**classification_report:** Se genera un resumen detallado de métricas por clase como precision, recall y f1-score.

**confusion_matrix:** Se genera la matriz de confusión, que muestra cómo se clasificaron realmente las clases frente a las predichas.

Con lo anterior tenemos que:

Precision: ¿Qué porcentaje de las predicciones positivas fueron correctas?

Recall: ¿Qué porcentaje de los verdaderos positivos fueron correctamente predichos?

F1-score: Media armónica entre precision y recall.

Support: Número real de muestras en cada clase.

In [16]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred_tree = modelo_tree.predict(X_test)

acc_tree = accuracy_score(y_test, y_pred_tree)
print(f"Árbol de Decisión - Accuracy: {acc_tree:.2f}")

print("\nMatriz de Confusión - Árbol de Decisión:")
print(confusion_matrix(y_test, y_pred_tree))

print("\nReporte de Clasificación - Árbol de Decisión:")
print(classification_report(y_test, y_pred_tree, target_names=['Solo Daños', 'Con Heridos', 'Con Muertos']))

Árbol de Decisión - Accuracy: 0.71

Matriz de Confusión - Árbol de Decisión:
[[24075   911   201]
 [ 5526  2997  4498]
 [  154    68   377]]

Reporte de Clasificación - Árbol de Decisión:
              precision    recall  f1-score   support

  Solo Daños       0.81      0.96      0.88     25187
 Con Heridos       0.75      0.23      0.35     13021
 Con Muertos       0.07      0.63      0.13       599

    accuracy                           0.71     38807
   macro avg       0.55      0.61      0.45     38807
weighted avg       0.78      0.71      0.69     38807



In [17]:
param_grid = {
    'max_depth': [3, 5, 10, 15, None],  # Profundidad máxima
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras para dividir
    'min_samples_leaf': [1, 2, 4],  # Mínimo número de muestras por hoja
    'max_features': ['auto', 'sqrt', 'log2', None],  # Número de características a considerar
    'criterion': ['gini', 'entropy'],  # Criterio para dividir
}

grid_search = GridSearchCV(estimator=modelo_tree, param_grid=param_grid,
                           cv=5,  # Validación cruzada de 5 pliegues
                           n_jobs=-1,  # Utiliza todos los núcleos disponibles
                           scoring='accuracy',  # Usa accuracy como métrica
                           verbose=1)

grid_search.fit(X_train, y_train)

print("Mejores hiperparámetros encontrados: ", grid_search.best_params_)

mejor_modelo = grid_search.best_estimator_

y_pred_mejor = mejor_modelo.predict(X_test)

# Evaluar el rendimiento del modelo ajustado
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print(f"Accuracy con los mejores hiperparámetros: {accuracy_score(y_test, y_pred_mejor):.2f}")

print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_mejor))

print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_mejor, target_names=['Solo Daños', 'Con Heridos', 'Con Muertos']))

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


/home/edwmacias/code/siniestrosBogota/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
450 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
208 fits failed with the following error:
Traceback (most recent call last):
  File "/home/edwmacias/code/siniestrosBogota/.venv/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/edwmacias/code/siniestrosBogota/.venv/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/home/edwmacias/code/siniestrosBogota/.venv/lib/python3.12/site-packages/sklearn/bas

Mejores hiperparámetros encontrados:  {'criterion': 'gini', 'max_depth': 15, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 5}
Accuracy con los mejores hiperparámetros: 0.73

Matriz de Confusión:
[[22926  1079  1182]
 [ 5223  5142  2656]
 [  136   250   213]]

Reporte de Clasificación:
              precision    recall  f1-score   support

  Solo Daños       0.81      0.91      0.86     25187
 Con Heridos       0.79      0.39      0.53     13021
 Con Muertos       0.05      0.36      0.09       599

    accuracy                           0.73     38807
   macro avg       0.55      0.55      0.49     38807
weighted avg       0.79      0.73      0.73     38807



In [18]:
!pip install imbalanced-learn

In [19]:
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [20]:
print(f"Distribución después de SMOTE: {Counter(y_train_smote)}")

Distribución después de SMOTE: Counter({0: 100747, 1: 100747, 2: 100747})


In [21]:
modelo_tree_smote = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
modelo_tree_smote.fit(X_train_smote, y_train_smote)

DecisionTreeClassifier(class_weight='balanced', max_depth=5, random_state=42)

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
y_pred_smote = modelo_tree_smote.predict(X_test)

acc_tree = accuracy_score(y_test, y_pred_smote)
print(f"SMOTE - Accuracy: {acc_tree:.2f}")

print("Matriz de Confusión SMOTE:")
print(confusion_matrix(y_test, y_pred_smote))

print("Reporte de Clasificación SMOTE:")
print(classification_report(y_test, y_pred_smote, target_names=['Solo Daños', 'Con Heridos', 'Con Muertos']))

SMOTE - Accuracy: 0.74
Matriz de Confusión SMOTE:
[[24072  1023    92]
 [ 5526  4523  2972]
 [  154   176   269]]
Reporte de Clasificación SMOTE:
              precision    recall  f1-score   support

  Solo Daños       0.81      0.96      0.88     25187
 Con Heridos       0.79      0.35      0.48     13021
 Con Muertos       0.08      0.45      0.14       599

    accuracy                           0.74     38807
   macro avg       0.56      0.58      0.50     38807
weighted avg       0.79      0.74      0.73     38807

